# Loading the Preprocessed Data

In [2]:
import os
import numpy as np

# Load the arrays
X_train = np.load('Train Test Dataset/X_train.npy')
X_test = np.load('Train Test Dataset/X_test.npy')
y_train = np.load('Train Test Dataset/y_train.npy')
y_test = np.load('Train Test Dataset/y_test.npy')

<div style="background-color: #cce5ff; padding: 10px; border: 1px solid #0066cc;">
    <h2 style="color: #0066cc; font-weight: bold;">Deep Learning Architecture - GRU</h2>
    
</div>


# Importing Libraries - Architecture

In [3]:
import time
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.optimizers import Adam
from tensorflow.keras.layers import GRU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, LeakyReLU

### Reshaping X_train for LSTM

In [6]:
X_train_array = X_train  
X_train_reshaped = X_train_array.reshape(X_train_array.shape[0], X_train_array.shape[1], 1)
input_shape = (X_train_reshaped.shape[1], X_train_reshaped.shape[2])

In [7]:
input_shape

(5, 1)

# 1. Network Initialization

In [8]:
batch_size = 700  
epochs = 500  
learning_rate = 0.001

# 2. Define Architecture

In [9]:
gru_model = Sequential([
    
    GRU(512, input_shape=input_shape, activation='relu', return_sequences=True),
    Dropout(0.1),
    GRU(256, activation='relu'),
    Dropout(0.1),
    Dense(1, activation='sigmoid')

])

In [10]:
gru_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru (GRU)                   (None, 5, 512)            791040    
                                                                 
 dropout (Dropout)           (None, 5, 512)            0         
                                                                 
 gru_1 (GRU)                 (None, 256)               591360    
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense (Dense)               (None, 1)                 257       
                                                                 
Total params: 1,382,657
Trainable params: 1,382,657
Non-trainable params: 0
_________________________________________________________________


# 3. Model Compilation

In [11]:
gru_model.compile(optimizer=Adam(learning_rate=learning_rate), loss='binary_crossentropy', metrics=['accuracy'])

<div style="background-color: #cce5ff; padding: 10px; border: 1px solid #0066cc;">
    <h2 style="color: #0066cc; font-weight: bold;">Integrating Callbacks</h2>
    
</div>


## EarlyStopping Callback

In [12]:
from keras.callbacks import EarlyStopping

In [13]:
early_stopping = EarlyStopping(monitor='val_loss', patience=30, restore_best_weights=True)

## Model Checkpoints Callback

In [14]:
import os
from keras.models import load_model
from keras.callbacks import ModelCheckpoint

In [15]:
checkpoint_path = '/Model Checkpoints/GRU_Model_Checkpoint.h5'

### Check if the checkpoint file exists

In [18]:
if os.path.exists(checkpoint_path):
    print("Loading model from checkpoint...")
    model = load_model(checkpoint_path)
else:
    print("Checkpoint not found. Initializing new model...")

Checkpoint not found. Initializing new model...


### Create the checkpoint callback

In [19]:
checkpoint = ModelCheckpoint(
    filepath=checkpoint_path, 
    monitor='val_loss', 
    verbose=1, 
    save_best_only=True, 
    mode='min'
)

# 4. Fit Network

In [20]:
start_time = time.time()

history = gru_model.fit(X_train_reshaped, y_train,
                         batch_size=batch_size,
                         epochs=epochs,
                         validation_split=0.1,
                         callbacks=[checkpoint, early_stopping], 
                         verbose=1)

end_time = time.time()

training_time = end_time - start_time
print("Training Time: ", training_time, "seconds")

Epoch 1/500


KeyboardInterrupt: 

# 5. Model Evaluation

# Importing Libraries - Evaluation

In [ ]:
import seaborn as sns
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# Plotting Loss and Accuracy Curve

### Loss Curve

In [ ]:
plt.figure(figsize=[8,6])

plt.plot(history.history['loss'], 'r', linewidth=3.0)
plt.plot(history.history['val_loss'], 'b', linewidth=3.0)

plt.xlabel('Epochs', fontsize=16)
plt.ylabel('Loss', fontsize=16)

plt.legend(['Training Loss', 'Validation Loss'], fontsize=18)
plt.title('Loss Curve of LSTM Neural Network', fontsize=16)

plt.grid(True)
plt.show()

### Accuracy Curve

In [ ]:
plt.figure(figsize=[8,6])

plt.plot(history.history['accuracy'], 'r', linewidth=3.0)
plt.plot(history.history['val_accuracy'], 'b', linewidth=3.0)

plt.xlabel('Epochs', fontsize=16)
plt.ylabel('Accuracy', fontsize=16)

plt.legend(['Training Accuracy', 'Validation Accuracy'], fontsize=18)
plt.title('Accuracy Curve of LSTM Neural Network', fontsize=16)

plt.grid(True)
plt.show()

### Reshaping X_test for LSTM Model Evaluation

In [ ]:
X_test_array = X_test.values
X_test_reshaped = X_test_array.reshape(X_test_array.shape[0], X_test_array.shape[1], 1)

In [ ]:
X_test_reshaped

###  Make predictions on test data

In [ ]:
y_pred_prob = gru_model.predict(X_test_reshaped)
y_pred = (y_pred_prob > 0.5).astype(int).flatten()  # Convert probabilities to classes

# Classification Report

In [ ]:
test_loss, test_accuracy = gru_model.evaluate(X_test_reshaped, y_test, verbose=0)
print(f'Test accuracy: {test_accuracy:.2f}, Test loss: {test_loss:.2f}')

In [ ]:
classification_report = classification_report(y_test, y_pred)
print(classification_report)

# Confusion Matrix

In [ ]:
class_labels = ["Alert", "Drowsy"]

conf_matrix = confusion_matrix(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred) * 100

# Plot confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', cbar=True, xticklabels=class_labels, yticklabels=class_labels, linewidths=.5)

plt.title(f'Confusion Matrix (Accuracy: {accuracy:.2f}%)', fontsize=16)

plt.xlabel('Predicted Label', fontsize=14)
plt.ylabel('True Label', fontsize=14)

plt.xticks(rotation=0, fontsize=12)  
plt.yticks(rotation=0, fontsize=12)  

plt.show()

# Evaluation Metrics

In [ ]:
# Calculate accuracy, precision, recall, F1-score, AUC-ROC

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
auc_roc = roc_auc_score(y_test, y_pred_prob)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)
print("AUC-ROC:", auc_roc)

# Receiver Operating Characteristic (ROC) Curve

In [ ]:
roc_data = {}

fpr, tpr, thresholds = roc_curve(y_test, y_pred_prob)
roc_auc = auc(fpr, tpr)
roc_data['LSTM Model'] = {'fpr': fpr, 'tpr': tpr, 'roc_auc': roc_auc}

# Plotting ROC Curves 
plt.figure(figsize=(10, 8))
for name, data in roc_data.items():
    plt.plot(data['fpr'], data['tpr'], lw=2, label=f'{name} (AUC = {data["roc_auc"]:.2f})')

plt.plot([0, 1], [0, 1], linestyle='--', color='gray', lw=2, label='Random Guessing')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate', fontsize=14)
plt.ylabel('True Positive Rate', fontsize=14)
plt.title('Receiver Operating Characteristic (ROC) Curve', fontsize=16)
plt.legend(loc='lower right', fontsize=12)
plt.grid(True)
plt.show()

# Saving the model and training history 

In [ ]:
gru_model.save('EEG_Driver_Drowsiness_GRU_Architecture.h5')
np.save('EEG_Driver_Drowsiness_GRU_history.npy', history.history)